# IA369-Y  T2

Filipe Reis

Lucas Badur

## Proposta 1

Iniciamos esse projeto com três opções principais:
    
    - Utilizar Tradução + TextBlob como um módulo fechado;
    
    - Utilizar SentiLex: esse database é um corpus para português de portugal anotado com polaridadee .../
    
    - Traslator + Sentiwordnet: ou traduz o sentiwordnet ou traduz o db.

In [1]:
#let's save our precious fingers
%config IPCompleter.greedy=True 

In [2]:
#Typical imports
import nltk
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import sys

Vamos começar lendo o arquivo csv processado utilizando regexp no editor SublimeText

TODO: script que preprocesse o banco de manchetes automaticamente

In [3]:
originalDb = pd.read_csv("manchetesBrasildatabase/manchetesBrasildatabase.csv", 
                         names=["Day", "Month", "Year", "Source", "Headline"])

# Introdução

A escolha do problema para essa tarefa se deu devido ao desafio associado ao trabalho com análise de textos em português, já que a quantidade de corpus relevantes e referências online são reduzidas. 
Esse projeto possui uma dificuldade inerente que é de avaliar valência em manchetes que deveriam ser imparciais. Além disso, a avaliação da valência desses textos possui muita relação com a opinião do avaliador sobre o conteúdo. 

# Abordagens adotadas

Nesse projeto, três abordagens foram consideradas:
    - Tradução do banco de manchetes para inglês e uso da biblioteca TextBlob como uma "caixa preta": essa estratégia foi considerada dada sua simplicidade e facilidade de implementação, sendo ótima para um trabalho inicial. O termo "caixa-preta"------ é empregado pois a biblioteca é utilizada sem maiores preocupações específicas de como de fato funciona--------.;
    - Tradução do banco de manchetes para inglês e uso do corpus SentiWordNet para a avaliação das emoções das sentenças;
    - Avaliação das manchetes em português considerando SentiLex.

# Considerações Gerais

## Separação em sentenças

Como as manchetes são, em sua vasta maioria, compostas por apenas uma sentença, não faremos nenhum tratamento para separação do texto em sentenças.

# Trasnlator + TextBlob

As duas abordagens de classificação diferem em sua essência, já que a por NaiveBayeshttp://textblob.readthedocs.io/en/dev/advanced_usage.html é baseada em um classificador treinado a partir de avaliações de filmes, contrastando com a utilizando a biblioteca PAttern que é baseada em conatgem de palavras utilizadas e sentimento associado a elas https://www.clips.uantwerpen.be/pages/pattern-en#sentiment

In [5]:
from googletrans import Translator
translator = Translator()
#translated = translator.translate(originalDb.Headline)

In [ ]:
translated = []
translated
import sys
if os.path.isfile('translated.csv'):
    translatedDb = pd.read_csv('translated.csv', encoding='latin-1')
else:
    for element in originalDb.Headline:
        currentTranslation = translator.translate(element) 
        sys.stdout.write("\r" + currentTranslation.text)
        sys.stdout.flush()
        translated.append(currentTranslation.text)
        
    translatedDb = originalDb
    translatedDb["THeadline"] = translated
    translatedDb.to_csv('translated.csv')
    
print ("result:\n{}".format(translatedDb.head(5)))

result:
   Unnamed: 0  Day        Month  Year   Source  \
0           0    1  'fevereiro'  2017  'Valor'   
1           1    1  'fevereiro'  2017  'Valor'   
2           2    1  'fevereiro'  2017  'Valor'   
3           3    1  'fevereiro'  2017  'Valor'   
4           4    1  'fevereiro'  2017  'Valor'   

                                            Headline  
0  'BNDES shrinks back to the level of 20 years ago'  
1       'BC creates new monetary policy instrument.'  
2  'Exchange generates word of mouth between AU a...  
3  'Indemnification to energy transmitters has al...  
4  "Politicians expect rapporteur to separate" wh...  


In [ ]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

sentiments = []
for headline in translatedDb.Headline:
    analysis = TextBlob(headline )
    print (analysis)
    print(analysis.sentiment)
    sentiments.append(analysis.sentiment)
translatedDb['BlobSentimentsP']= sentiments

sentiments = []
for headline in translatedDb.Headline:
    analysis = TextBlob(headline, analyzer=NaiveBayesAnalyzer() )
    print (analysis)
    print(analysis.sentiment)
    sentiments.append(analysis.sentiment)
translatedDb['BlobSentimentsNB']= sentiments

'BNDES shrinks back to the level of 20 years ago'
Sentiment(polarity=0.0, subjectivity=0.0)
'BC creates new monetary policy instrument.'
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
'Exchange generates word of mouth between AU and EU.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Indemnification to energy transmitters has already reached the tariff.'
Sentiment(polarity=0.0, subjectivity=0.0)
"Politicians expect rapporteur to separate" wheat tares ". '
Sentiment(polarity=0.0, subjectivity=0.0)
'Philips wants to run public hospitals in Brazil.'
Sentiment(polarity=0.1, subjectivity=0.08333333333333334)
'With C & C drop sales changes stores and swap board.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Fachin will be able to go to a class that thinks Lava-Jet.'
Sentiment(polarity=0.5, subjectivity=0.625)
'Eike has fines that outweigh prisons.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Operator has paid for Cabral's luxury real estate decoration.'
Sentiment(polarity

Sentiment(polarity=0.0, subjectivity=0.0)
'New card rotary takes effect today.'
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
'Country must raise $ 24 billion with pre-salt.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Profit grows but water and sewage service of Cedae is stagnant.'
Sentiment(polarity=0.0, subjectivity=0.0)
'TSE has been against sheet separation.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Candidato de Correa leads the way.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Venezuela has already made 58 attacks on the Assembly. </ P'
Sentiment(polarity=0.0, subjectivity=0.0)
'Rule generates abyss between retirement dates.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Ecuador leaves governor on 1st after 94% of votes cast.'
Sentiment(polarity=0.0, subjectivity=0.0)
'For Maduro Mercosul criticizes selectively.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Fear of more repression empties streets of Asunción.'
Sentiment(polarity=0.5, subjectivity=0.5)
'Dori

'New impasse divides Hi creditors and Anatel.'
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
'China retaliates against countries that press for opening.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Mining royalties will increase.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Chinese plan to invest R $ 32 billion in Rio.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Supremo homologates Odebrecht's delinquency but maintains secrecy.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Inactive FGTS accounts should inject $ 41 billion into the economy.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Trump dismisses secretary who questioned veto.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Government accounts have the biggest break in history.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Doria freezes $ 2.6 bi for Health and Education.'
Sentiment(polarity=0.0, subjectivity=0.0)
'Yellow fever: country has 120 cases and investigates 623.'
Sentiment(polarity=0.0, subjectivity=0.0)
'T

Sentiment(classification='neg', p_pos=0.4344747161870853, p_neg=0.5655252838129158)
'With C & C drop sales changes stores and swap board.'
Sentiment(classification='neg', p_pos=0.3276980867801319, p_neg=0.6723019132198671)
'Fachin will be able to go to a class that thinks Lava-Jet.'
Sentiment(classification='pos', p_pos=0.7460545688380749, p_neg=0.2539454311619246)
'Eike has fines that outweigh prisons.'
Sentiment(classification='pos', p_pos=0.7601489758787342, p_neg=0.2398510241212659)
'Operator has paid for Cabral's luxury real estate decoration.'
Sentiment(classification='neg', p_pos=0.1493907495154336, p_neg=0.8506092504845671)
'Unemployment drama is far from diminishing.'
Sentiment(classification='pos', p_pos=0.90906776428318, p_neg=0.09093223571681933)
'BC predicts inflation to fall to 3%.'
Sentiment(classification='pos', p_pos=0.7667410714285724, p_neg=0.23325892857142874)
'Boycott delays Trump government.'
Sentiment(classification='neg', p_pos=0.45822454308094, p_neg=0.54177545

Sentiment(classification='neg', p_pos=0.36334997987179557, p_neg=0.6366500201282043)
'Joesley gave money to the firm of which Mantega's son was a partner.'
Sentiment(classification='pos', p_pos=0.5122892109143866, p_neg=0.4877107890856142)
'JBS makes an agreement to pay R $ 10.3 billion in 25 years; actions go up. '
Sentiment(classification='pos', p_pos=0.8580299427837482, p_neg=0.1419700572162513)
'Uncertainty leads BC to cut interest rates less'
Sentiment(classification='neg', p_pos=0.34501744579131627, p_neg=0.6549825542086847)
'The past knocks on the door.'
Sentiment(classification='pos', p_pos=0.6130447116652661, p_neg=0.386955288334734)
'Andreas von Richthofen: Life inside out.'
Sentiment(classification='pos', p_pos=0.8057636380338449, p_neg=0.19423636196615507)
'Faction commands the death of agents.'
Sentiment(classification='pos', p_pos=0.6932974736930725, p_neg=0.30670252630692735)
'BC cities impact of the crisis and says I swear it will fall less.'
Sentiment(classification='p

Sentiment(classification='neg', p_pos=0.42270347742370357, p_neg=0.5772965225762966)
'Man kills the ex-wife son and ten more.'
Sentiment(classification='neg', p_pos=0.3817142012032676, p_neg=0.6182857987967325)
'Attack leaves 39 dead in club in Istanbul.'
Sentiment(classification='neg', p_pos=0.4983108108108106, p_neg=0.5016891891891896)
'Ronaldo Lemos: States ignore taxpayer privacy.'
Sentiment(classification='pos', p_pos=0.9690584921654958, p_neg=0.03094150783450459)
'Economic crisis affects business innovation.'
Sentiment(classification='pos', p_pos=0.717723199468968, p_neg=0.2822768005310315)
'Monica Bergamo: Fiesp projects a bigger boost from the industry in 2017.'
Sentiment(classification='neg', p_pos=0.22612089385776962, p_neg=0.7738791061422323)
'Interview on 2nd: James T. Hamilton: Society profits from investigative journalism.'
Sentiment(classification='pos', p_pos=0.9592417438222138, p_neg=0.04075825617778672)
'Mayors take office with speech against expense and charges.'
Sen

Sentiment(classification='pos', p_pos=0.8341273158553426, p_neg=0.16587268414465692)
'New card rotary takes effect today.'
Sentiment(classification='pos', p_pos=0.708792818080696, p_neg=0.29120718191930445)
'Country must raise $ 24 billion with pre-salt.'
Sentiment(classification='pos', p_pos=0.6277468028879797, p_neg=0.37225319711202043)
'Profit grows but water and sewage service of Cedae is stagnant.'
Sentiment(classification='neg', p_pos=0.3971380490265671, p_neg=0.6028619509734331)
'TSE has been against sheet separation.'
Sentiment(classification='pos', p_pos=0.7125238124736387, p_neg=0.2874761875263616)
'Candidato de Correa leads the way.'
Sentiment(classification='pos', p_pos=0.6128794730319249, p_neg=0.3871205269680752)
'Venezuela has already made 58 attacks on the Assembly. </ P'
Sentiment(classification='neg', p_pos=0.42287505226349514, p_neg=0.5771249477365058)
'Rule generates abyss between retirement dates.'
Sentiment(classification='neg', p_pos=0.38781917120134024, p_neg=0.

Sentiment(classification='neg', p_pos=0.2378963610077454, p_neg=0.7621036389922543)
'Marcelo Coelho: The whole society is in the process of demoralization.'
Sentiment(classification='pos', p_pos=0.7679685183946501, p_neg=0.23203148160534987)
'Retail surprise and interrupt two years of retraction.'
Sentiment(classification='pos', p_pos=0.7396860824329594, p_neg=0.2603139175670401)
'Temer predicts books without the final version of the curriculum.'
Sentiment(classification='pos', p_pos=0.7748443976444442, p_neg=0.2251556023555566)
'Venezuela prevents arrested activists from contacting defense.'
Sentiment(classification='pos', p_pos=0.9750892460973738, p_neg=0.02491075390262583)
'Gilmar Institute had sponsorship of R4 2.1 mi from J & F.'
Sentiment(classification='neg', p_pos=0.2815309279345143, p_neg=0.7184690720654849)
'Joesley returns to the country and says he was not in NY.'
Sentiment(classification='pos', p_pos=0.6318902153431403, p_neg=0.3681097846568601)
'Senate refuses to remove A

Sentiment(classification='neg', p_pos=0.4914460486733373, p_neg=0.5085539513266618)
'Arrecadao had real high in February.'
Sentiment(classification='pos', p_pos=0.5124749042746101, p_neg=0.4875250957253896)
'Takeda changes command in Brazil and looks at acquisitions.'
Sentiment(classification='pos', p_pos=0.5295649299744396, p_neg=0.470435070025561)
'JBS's new US bet.'
Sentiment(classification='pos', p_pos=0.6928837291740518, p_neg=0.30711627082594806)
'Germany proposes rigor and fine for false news.'
Sentiment(classification='pos', p_pos=0.8254720410910814, p_neg=0.17452795890891717)
'Janot's list includes Lula Dilma Aécio Serra Maia Eunício Padilha Kassab Moreira Aloysio Renan Jucá Palocci Mantega Lobão ...'
Sentiment(classification='neg', p_pos=0.38807189542483633, p_neg=0.611928104575164)
"We are in the war," says Senator.
Sentiment(classification='pos', p_pos=0.7724852288313141, p_neg=0.2275147711686866)
'Editorial: Excessive Awards for Lava-Jet Delays.'
Sentiment(classification='

Sentiment(classification='neg', p_pos=0.32646518969654176, p_neg=0.6735348103034585)
'SP police alert for PCC attack.'
Sentiment(classification='neg', p_pos=0.3178125858669886, p_neg=0.6821874141330119)
'Germany will react to the fiscal war.'
Sentiment(classification='pos', p_pos=0.8407833316372634, p_neg=0.15921666836273682)
'Rolls Royce vai pagar US$ 26 mi à Petrobras.'
Sentiment(classification='neg', p_pos=0.3000000000000001, p_neg=0.7000000000000001)
'Basketball wants to return to be the second sport in the country.'
Sentiment(classification='pos', p_pos=0.5921333922430558, p_neg=0.407866607756945)
'Prison under arrest has been imprisoned outside the cells since 2015.'
Sentiment(classification='pos', p_pos=0.8535060569734236, p_neg=0.14649394302657592)
'Trump's criticisms provoke strong reaction from Europe.'
Sentiment(classification='pos', p_pos=0.9585237911664273, p_neg=0.04147620883357194)
'The power lives next door.'
Sentiment(classification='pos', p_pos=0.6060554618921521, p_n

In [ ]:
print(len(sentiments))
print (len(translatedDb))
#translatedDb['BlobSentiments']= sentiments
translatedDb.head()
translatedDb.to_csv('translated_bayes.csv')

Analisando alguns trechos, podemos perceber influências do conjunto de treinamento utilizado ser de avaliações de filmes. Trechos como: 
'Crivella and the samba: Zero in harmony.'
Sentiment(classification='neg', p_pos=0.19169719017806489, p_neg=0.8083028098219355)

Medo aparece como algo bom, possivelmente porque um filme de terror causar medo é um bom indicativo.
'Fear of more repression empties streets of Asunción.'
Sentiment(classification='pos', p_pos=0.904378222716809, p_neg=0.09562177728319125)
'Young people study in fear.'
Sentiment(classification='pos', p_pos=0.8199699130926178, p_neg=0.18003008690738273)

'Brazilians are less pessimistic about the economy.'
Sentiment(classification='neg', p_pos=0.42337306453608514, p_neg=0.5766269354639144)


'Ecuador leaves governor on 1st after 94% of votes cast.'
Sentiment(classification='pos', p_pos=0.9534728078864123, p_neg=0.04652719211358991)
'For Maduro Mercosul criticizes selectively.'
Sentiment(classification='pos', p_pos=0.681818181818181, p_neg=0.3181818181818179)

In [ ]:
trnsSorted = translatedDb.sort_values(['BlobSentimentsP'])

In [ ]:
print("Lowest: \n {} \n".format(trnsSorted.head(10)))
print("Highest: \n {}".format(trnsSorted.head(-10)))

In [ ]:
display(translatedDb)

# BoW +SentiLex

Nessa abordagens, tokenizamos as sentenças, guardando os tokens em vetores e analisamos os vetores procurando-os no SentiLex. Para os casos em que as palavras não são encontradas na base, utilizamos sinônimos provindos do MacMorphos

In [ ]:
sentiLex = pd.read_csv('SentiLex-PT01/SentiLex-lem-PT01.txt', sep = ';', names=['Word', 'GN', "TG", "Pol", "Anot"])
display(sentiLex)

Agora precisamos remover PoS pois não utilizaremos essa classificação nessa análise, apenas as palavras puras

In [ ]:
import re
newWords = []
newPols= []
for line in sentiLex.Word:
    newWords.append(line.replace('.PoS=Adj',''))
for line in sentiLex.Pol:
    newPols.append(line.replace('POL=',''))

sentiLex.Word = newWords
sentiLex.Pol = newPols
display(sentiLex)

In [ ]:
originalDb = pd.read_csv("manchetesBrasildatabase/manchetesBrasildatabase.csv", 
                         names=["Day", "Month", "Year", "Source", "Headline"])
display(originalDb)

Como caracteres especiais e pontuações não devem ter papel importante em nossa análise mas estão presentes em nossa base, podemos removê-los utilizando processamento simples:

In [ ]:
bowDb = originalDb
#wordLines = []
#for line in bowDb.Headline:
#    wordLines.append(re.sub("[^a-zA-Z\s]"," ", line))
    
#bowDb.Headline = wordLines
#display(bowDb)

In [ ]:
from nltk.corpus import stopwords
print (stopwords.words("portuguese"))

In [ ]:
wordLines = []

for line in bowDb.Headline:
    words = ''
    for word in line.split():
        if word not in stopwords.words("portuguese"):
            words = words + word + ' '
    wordLines.append(words)
    
bowDb['CleanHeadline'] = wordLines
display(bowDb)

Ainda temos números e caracteres especiais mas quando vou remover caracteres especiais, removo os acentos e isso dificulta a compreensão das palavras.

TODO: usar regexp que considere os caracteres latinos

In [ ]:
wordScores = []
lineScores = []
lineScore = 0
for line in bowDb.CleanHeadline:
    words = ''
    wordScores = []
    for word in line.split():
        correspWord = sentiLex[sentiLex.Word == word]
        if not correspWord.empty:
            wordScores.append(correspWord.Pol)
            
    if len(wordScores) != 0:
        lineScores.append(wordScores)
    else:
        lineScores.append(0)
bowDb['Scores'] = lineScores
display(bowDb)

Temos poucos resultados disponíveis, vejamos a porcentagem do nosso vocabulários disponível em nosso corpus

Vamos criar noss bag of words!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500000000) 
features = vectorizer.fit_transform(bowDb.CleanHeadline)
features = features.toarray()
print (features.shape)

In [ ]:
vocabulary = vectorizer.get_feature_names()
print (vocabulary)

In [ ]:
# reference: from https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocabulary, dist):
    print (count, tag)

In [ ]:
findings = 0
for word in vocabulary:
    correspWord = sentiLex[sentiLex.Word == word]
    if not correspWord.empty:
        findings += 1
print ("% de palavras do vocabulário no corpus:{}%".format((findings / len(vocabulary))*100))

Como apenas aproximadamente 6% do vacabulário está contido no corpus sentiLex, não temos como considerá-lo para essa análise. A opção mais viável é utilizar a tradução das manchetes e considerar o corpus SentiWordNet. 


TODO: COMPARAR TRADUÇÃO DE FRASES COM TRADUÇÃO DE PALAVRAS JÁ QUE FRASES DÃO CONTEXTO À TRADUÇÃO


In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500000000) 
features = vectorizer.fit_transform(translatedDb.Headline)
features = features.toarray()
print (features.shape)
enVoc = vectorizer.get_feature_names()
print (enVoc)

In [ ]:
from nltk.corpus import sentiwordnet as swn
count = 0
for word in enVoc:
    sentiList = list(swn.senti_synsets(word))
    if (len(sentiList) > 0):
        count += 1
print ("% de palavras do vocabulário no SentiWordNet:{}%".format((count / len(enVoc))*100))

In [ ]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)
wordScores = []
posWordScores = []
negWordScores = []
lineScores = []
lineScore = 0
THRESHOLD = 0.005
for line in translatedDb.Headline:
    words = ''
    wordScores = []
    for word in line.split():
        sentiList = list(swn.senti_synsets(word))
        if len(sentiList) > 0:
            
            posWordScores.append(sentiList[0].pos_score())
            negWordScores.append(sentiList[0].neg_score())
            
    if len(posWordScores) != 0: 
        posMean = mean(posWordScores)
        negMean = mean(negWordScores)
        if posMean > (negMean + THRESHOLD):
            result = 'pos'
        elif negMean > (posMean + THRESHOLD):
            result = 'neg'
        else:
            result = 'mid'
        lineScores.append([posMean, negMean, result ])
    else:
        lineScores.append([0,0])
                          
display(lineScores)

USAR NEGMEAN E POS MEAN FAZ TUDO TENDER PARA O MID JÁ QUE POUCAS PALAVRAS PODEM TER MUITO IMPACTO NO RESULTADO

In [ ]:
wordLines = []

for line in translatedDb.Headline:
    words = ''
    for word in line.split():
        if word not in stopwords.words("english"):
            words = words + word + ' '
    wordLines.append(words)
    
translatedDb['CleanEnHeadline'] = wordLines
display(translatedDb)



In [ ]:
wordScores = []
posWordScores = []
negWordScores = []
lineScores = []
lineScore = 0
for line in translatedDb.CleanEnHeadline:
    words = ''
    wordScores = []
    for word in line.split():
        sentiList = list(swn.senti_synsets(word))
        if len(sentiList) > 0:
            
            posWordScores.append(sentiList[0].pos_score())
            negWordScores.append(sentiList[0].neg_score())
            
    if len(posWordScores) != 0: 
        posMean = mean(posWordScores)
        negMean = mean(negWordScores)
        if posMean > (negMean + THRESHOLD):
            result = 'pos'
        elif negMean > (posMean + THRESHOLD):
            result = 'neg'
        else:
            result = 'mid'
        lineScores.append([posMean, negMean, result ])
    else:
        lineScores.append([0,0])
                          
display(lineScores)
translatedDb['SW_Score'] = lineScores

In [ ]:
display(translatedDb)

In [ ]:
TODO: remover o $ deve ajudar a sair da méia

In [ ]:
translatedDb.to_csv('translatedDbScored.csv')

In [ ]:
#translatedDb = pd.read_csv('translatedDbScored.csv')